In [ ]:
import cv2 as cv
import pandas as pd
import numpy as np
import tensorflow as tf
import os
haar_cascade=cv.CascadeClassifier(cv.data.haarcascades + "haarcascade_frontalface_default.xml")
people = ['BenAfflek','Madonna']
DIR = 'train'


def create_train():
    features = []
    labels = []
    for person in people:
        path = f"{DIR}/{person}"
        label ="BenAfflek" if people.index(person)==0 else "Madonna"

        for img in os.listdir(path):
            img_path = f"{path}/{img}"
            if(img_path is None):
                print("doldum")
            image_seq=cv.imread(img_path)
            
            
            gray=cv.cvtColor(image_seq,cv.COLOR_BGR2GRAY)
            faces_rect = haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)

            for (x,y,w,h) in faces_rect:
                faces_roi =cv.resize(image_seq[y:y+h, x:x+w],(224,224))
                  
                features.append(faces_roi)
                labels.append(label)
            
            
            

            fe
    return np.array(features,dtype="float32")/255.0,np.array(labels)

features,labels=create_train()
print('Training done ------------dd---')
print(features)
print(labels)

Training done ------------dd---
[[[[0.21960784 0.33333334 0.57254905]
   [0.21960784 0.33333334 0.57254905]
   [0.19215687 0.29803923 0.5137255 ]
   ...
   [0.05098039 0.09019608 0.10196079]
   [0.04705882 0.09019608 0.10588235]
   [0.04705882 0.09019608 0.10588235]]

  [[0.21960784 0.33333334 0.57254905]
   [0.21960784 0.33333334 0.57254905]
   [0.19215687 0.29803923 0.5137255 ]
   ...
   [0.05098039 0.09019608 0.10196079]
   [0.04705882 0.09019608 0.10588235]
   [0.04705882 0.09019608 0.10588235]]

  [[0.21176471 0.32156864 0.56078434]
   [0.21176471 0.32156864 0.56078434]
   [0.1882353  0.29411766 0.50980395]
   ...
   [0.03529412 0.07450981 0.08627451]
   [0.03137255 0.07450981 0.09411765]
   [0.03137255 0.07450981 0.09411765]]

  ...

  [[0.00784314 0.07843138 0.05490196]
   [0.00784314 0.07843138 0.05490196]
   [0.00784314 0.07450981 0.05490196]
   ...
   [0.03137255 0.11372549 0.05490196]
   [0.03137255 0.11372549 0.05490196]
   [0.03137255 0.11372549 0.05490196]]

  [[0.       

In [2]:
labels=pd.get_dummies(labels)

In [3]:
data=tf.data.Dataset.from_tensor_slices((tf.constant(features),tf.constant(labels))).batch(4)
data

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.bool, name=None))>

In [4]:
def create_model():
    model=tf.keras.Sequential(
        [
            tf.keras.Input(shape=(224,224,3)),
            tf.keras.layers.Conv2D(32,kernel_size=(3,3),activation='relu'),
            tf.keras.layers.MaxPool2D(pool_size=(2,2)),
            tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
            tf.keras.layers.MaxPool2D(pool_size=(2,2)),
            tf.keras.layers.Conv2D(128,kernel_size=(3,3),activation='relu'),
            tf.keras.layers.MaxPool2D(pool_size=(2,2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(128,activation="relu"),
            tf.keras.layers.Dense(2,activation="softmax")
        ]
    )
    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(),
        metrics=["accuracy"]
    )
    return model
def train_model():
    model=create_model()
    model.fit(data,epochs=30)
    return model


In [5]:
model=train_model()

Epoch 1/30
6/6 [==============================] - 2s 65ms/step - loss: 11.2440 - accuracy: 0.5217
Epoch 2/30
6/6 [==============================] - 0s 62ms/step - loss: 1.0054 - accuracy: 0.4783
Epoch 3/30
6/6 [==============================] - 0s 59ms/step - loss: 0.7542 - accuracy: 0.4783
Epoch 4/30
6/6 [==============================] - 0s 60ms/step - loss: 0.6972 - accuracy: 0.4783
Epoch 5/30
6/6 [==============================] - 0s 62ms/step - loss: 0.6920 - accuracy: 0.4783
Epoch 6/30
6/6 [==============================] - 0s 62ms/step - loss: 0.6748 - accuracy: 0.4783
Epoch 7/30
6/6 [==============================] - 0s 60ms/step - loss: 0.6761 - accuracy: 0.4783
Epoch 8/30
6/6 [==============================] - 0s 61ms/step - loss: 0.6381 - accuracy: 0.6522
Epoch 9/30
6/6 [==============================] - 0s 59ms/step - loss: 0.6162 - accuracy: 0.7826
Epoch 10/30
6/6 [==============================] - 0s 62ms/step - loss: 0.5501 - accuracy: 0.7826
Epoch 11/30
6/6 [===========

In [6]:
val_dir="test"

In [7]:
os.listdir(val_dir)

['images.jpg',
 'Madonna_Rebel_Heart_Tour_2015_-_Stockholm_(23051472299)_(cropped).jpg']

In [19]:
labels.columns[0]

'BenAfflek'

In [19]:
def predict():
    features = []
    
    
    for img in os.listdir(val_dir):
        img_path = f"{val_dir}/{img}"
        print(img_path)
        
        image_seq=cv.imread(img_path)
        if image_seq is None:
            continue   
            
        gray=cv.cvtColor(image_seq,cv.COLOR_BGR2GRAY)
        faces_rect = haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=3)
        if faces_rect is None:
            continue   
        for (x,y,w,h) in faces_rect:
            faces_roi =cv.resize(image_seq[y:y+h, x:x+w],(224,224))
            pred=model.predict(np.expand_dims(faces_roi, axis=0) / 255.0)
            print(np.argmax(pred))
            
            cv.rectangle(image_seq, (x, y), (x + w, y + h), (0, 255, 0), thickness=2)
            cv.putText(image_seq, labels.columns[np.argmax(pred)], (50,50), cv.FONT_HERSHEY_COMPLEX, 1.0, (0,255,0), thickness=2)
            
            cv.imshow("Pred",image_seq)
            cv.waitKey(0)
            cv.destroyAllWindows()
    
predict()




test/images.jpg
1/1 [==============================] - 0s 38ms/step
0
test/Madonna_Rebel_Heart_Tour_2015_-_Stockholm_(23051472299)_(cropped).jpg
1/1 [==============================] - 0s 28ms/step
1
1/1 [==============================] - 0s 32ms/step
0


In [85]:
len(features_p)

3

In [86]:
labels

[1, 1, 1]

In [71]:
valid=tf.data.Dataset.from_tensor_slices(tf.constant(features_p)).batch(4)
valid

<BatchDataset element_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None)>

In [9]:
model.predict(valid)

NameError: name 'valid' is not defined

In [2]:
import numpy as np

In [3]:
np.argmax([9.9977928e-01, 2.2074059e-04])
       
       

0